In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity
from sklearn.neighbors import NearestNeighbors
from scipy.sparse import csr_matrix
from sklearn.decomposition import TruncatedSVD
from surprise import Reader, Dataset, SVD, SVDpp, SlopeOne, NMF, NormalPredictor, KNNBaseline, KNNBasic, KNNWithMeans, KNNWithZScore, BaselineOnly, CoClustering
from surprise.model_selection import cross_validate, train_test_split
from surprise import accuracy
from surprise.accuracy import rmse

import warnings
warnings.simplefilter('ignore')

%matplotlib inline

In [2]:
users = pd.read_csv('../input/users.csv')
posts = pd.read_csv('../input/posts.csv')
views = pd.read_csv('../input/views.csv')

In [3]:
users.rename(columns = {'_id':'user_id'},inplace=True)
# print(users.academics.unique())
# print(users.gender.unique())
# print(users.describe())
users.head()

,user_id,name,gender,academics
0,5d60098a653a331687083238,Nivesh Singh Chauhan,male,undergraduate
1,5d610ae1653a331687083239,Gaurav Sharma,male,graduate
2,5d618359fc5fcf3bdd9a0910,Akshay Mishra,male,undergraduate
3,5d6d2bb87fa40e1417a49315,Saksham Mathur,male,undergraduate
4,5d7c994d5720533e15c3b1e9,Varun Chowhan,male,undergraduate


In [4]:
posts.rename(columns = {'_id':'post_id',' post_type':'post_type'},inplace=True)
posts.category = posts.category.fillna('')
# print(posts.post_type.unique())
# print(posts.describe())
posts.head()

,post_id,title,category,post_type
0,5d62abaa65218653a132c956,hello there,Plant Biotechnology,blog
1,5d6d39567fa40e1417a4931c,Ml and AI,Artificial Intelligence|Machine Learning|Infor...,blog
2,5d7d23315720533e15c3b1ee,What is an Operating System ?,Operating Systems,blog
3,5d7d405e5720533e15c3b1f3,Lord Shiva,Drawings,artwork
4,5d80dfbc6c53455f896e600e,How Competition law evolved?,Competition Laws,blog


In [5]:
# print(views.describe())
views.head()

,user_id,post_id,timestamp
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z
1,5ed3748576027d35905ccaab,5ed4cbadbd514d602c1531a6,2020-06-01T09:39:20.021Z
2,5ed0defa76027d35905cc2de,5eac305f10426255a7aa9dd3,2020-06-01T08:12:42.682Z
3,5ed0defa76027d35905cc2de,5ed1ff0276027d35905cc60d,2020-06-01T08:10:23.880Z
4,5ed0defa76027d35905cc2de,5ed3820f76027d35905ccac8,2020-06-01T08:08:54.124Z


In [6]:
df = pd.merge(views,users,on='user_id')
df = pd.merge(df,posts,on='post_id')
# df.drop(columns=['name','timestamp'],inplace = True)
df.head()

,user_id,post_id,timestamp,name,gender,academics,title,category,post_type
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,2020-06-01T10:46:45.131Z,Niriksha Sharma,female,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
1,5d7c994d5720533e15c3b1e9,5ec821ddec493f4a2655889e,2020-05-22T20:11:32.317Z,Varun Chowhan,male,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
2,5e5af599d701ab08af792b63,5ec821ddec493f4a2655889e,2020-05-22T20:01:37.309Z,Ilupeju Ayokunnumi,female,graduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
3,5de50d768eab6401affbb135,5ec821ddec493f4a2655889e,2020-05-22T20:10:41.100Z,thesocialcomment,male,graduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork
4,5deeef6142a8854bf6eabab9,5ec821ddec493f4a2655889e,2020-05-22T20:08:45.061Z,siddharth saxena,male,undergraduate,Save Earth.,Visual Arts|Graphic Design|Artistic design|Gra...,artwork


# Problem 2 : Recommend similar posts for the given post

In [7]:
tf = TfidfVectorizer(analyzer='word',ngram_range=(1, 2),min_df=0, stop_words='english')
tfidf_matrix1 = tf.fit_transform(posts['title'])
tfidf_matrix2 = tf.fit_transform(posts['category'])
tfidf_matrix3 = tf.fit_transform(posts['post_type'])
cosine_sim1 = linear_kernel(tfidf_matrix1, tfidf_matrix1)
cosine_sim2 = linear_kernel(tfidf_matrix2, tfidf_matrix2)
cosine_sim3 = linear_kernel(tfidf_matrix3, tfidf_matrix3)
cosine_sim_Tfidf = (cosine_sim1 + cosine_sim2 + cosine_sim3)/3

In [8]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''
        
def create_mixture(x):
    return x['title1'] + ' ' + x['category1'] + ' ' + x['post_type1']

features = ['title', 'category', 'post_type']
for feature in features:
    posts[feature+'1'] = posts[feature].apply(clean_data)     
posts['mixture'] = posts.apply(create_mixture, axis=1)

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(posts['mixture'])
cosine_sim_count = cosine_similarity(count_matrix, count_matrix)

posts.drop(columns = ['title1','category1','post_type1','mixture'],inplace=True)

In [9]:
indices = pd.Series(posts.index, index=posts.title)
def get_recommendations(post,cosine_sim):
    idx = indices[post]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    sim_scores = sim_scores[1:31]
    post_indices = [i[0] for i in sim_scores]
    return posts.loc[post_indices]

In [10]:
get_recommendations('Ml and AI',cosine_sim_Tfidf).head()

,post_id,title,category,post_type
170,5e7c78fdcfc8b713f5ac7daa,Types Of AI.,Computer Technology|Artificial Intelligence,blog
224,5e897ab4a3258347b42f25ca,10 Best Artificial Intelligence (AI) Courses O...,Computer Technology|Machine Learning,blog
122,5e5bdf4fd701ab08af792bfd,Artificial Intelligence,Computer Technology|Artificial Intelligence,blog
186,5e81a4f2a3258347b42f21d3,7 Best Python Data Science Courses & Certifica...,Computer Technology|Machine Learning,blog
450,5ecd5d417023451e662235c5,"Machine Learning”&“Operations"" (MlOps)",Computer Technology|Machine Learning,blog


In [11]:
get_recommendations('Ml and AI',cosine_sim_count).head()

,post_id,title,category,post_type
122,5e5bdf4fd701ab08af792bfd,Artificial Intelligence,Computer Technology|Artificial Intelligence,blog
169,5e7bd922cfc8b713f5ac7da9,What sports will look like in the future,Computer Technology|Robotics|Data Science|Info...,blog
291,5e966ad3a3258347b42f2aaf,8 Best Machine Learning Courses for 2020,Computer Technology|Cloud Computing|Artificial...,blog
450,5ecd5d417023451e662235c5,"Machine Learning”&“Operations"" (MlOps)",Computer Technology|Machine Learning,blog
16,5dadf6ae610ba040fbfdf5c8,MIS,Computer Technology|Information Technology,blog


In [12]:
get_recommendations('Ml and AI',(cosine_sim_count + cosine_sim_Tfidf)/2).head()

,post_id,title,category,post_type
122,5e5bdf4fd701ab08af792bfd,Artificial Intelligence,Computer Technology|Artificial Intelligence,blog
170,5e7c78fdcfc8b713f5ac7daa,Types Of AI.,Computer Technology|Artificial Intelligence,blog
169,5e7bd922cfc8b713f5ac7da9,What sports will look like in the future,Computer Technology|Robotics|Data Science|Info...,blog
291,5e966ad3a3258347b42f2aaf,8 Best Machine Learning Courses for 2020,Computer Technology|Cloud Computing|Artificial...,blog
450,5ecd5d417023451e662235c5,"Machine Learning”&“Operations"" (MlOps)",Computer Technology|Machine Learning,blog


# Problem 1 : Recommend posts for the given user

In [13]:
f1 = {'artwork': 1, 
      'blog'  : 2, 
      'project' : 3, 
      'skill' : 4}

f2 = {'female' : 7, 
      'male' : 6, 
      'undefined' : 5}

f3 = {'undergraduate' : 10, 
      'graduate' : 9, 
      'undefined' : 8}

df['strength'] = (df['post_type'].apply(lambda x: f1[x]) + df['gender'].apply(lambda x: f2[x]) + df['academics'].apply(lambda x: f3[x]))/3
df['strength'] = 5*(df['strength'].values/max(df['strength'].values))

df = df[['user_id','post_id','strength']]
df.head()

,user_id,post_id,strength
0,5df49b32cc709107827fb3c7,5ec821ddec493f4a2655889e,4.285714
1,5d7c994d5720533e15c3b1e9,5ec821ddec493f4a2655889e,4.047619
2,5e5af599d701ab08af792b63,5ec821ddec493f4a2655889e,4.047619
3,5de50d768eab6401affbb135,5ec821ddec493f4a2655889e,3.809524
4,5deeef6142a8854bf6eabab9,5ec821ddec493f4a2655889e,4.047619


In [14]:
# pivot_matrix = pd.pivot_table(df,index='user_id', columns='post_id', values='strength')
# pivot_matrix = pivot_matrix.fillna(pivot_matrix.mean(axis=0))
# pivot_matrix.head()

In [15]:
reader = Reader()
data = Dataset.load_from_df(df, reader)
benchmark = []
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    results = cross_validate(algorithm, data, measures=['RMSE','MAE'], cv=5, verbose=False)
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Don

In [16]:
results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')
results

,test_rmse,test_mae,fit_time,test_time
Algorithm,,,,
KNNBaseline,0.123344,0.086764,0.005099,0.008548
KNNWithMeans,0.127091,0.086277,0.003858,0.007408
KNNWithZScore,0.135462,0.089567,0.009233,0.008427
SVDpp,0.161772,0.123687,1.892664,0.028057
BaselineOnly,0.179974,0.138511,0.003753,0.002729
SVD,0.189725,0.146582,0.137657,0.004896
KNNBasic,0.195983,0.147868,0.001343,0.006368
SlopeOne,0.214147,0.160043,0.014385,0.020247
NormalPredictor,0.360570,0.290513,0.003038,0.003454


In [17]:
# Using Alternating Least Squares (ALS)

sim_options = {
    "name": 'pearson_baseline' ,
    "user_based": True,
    'shrinkage': 0
}

# ['pearson_baseline','msd', 'cosine']
# ['als', 'sgd'],

bsl_options = {'method': 'als', 
#                'learning_rate': .00005,
#                'reg': 2,
#                'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }

algo = KNNBaseline(sim_options=sim_options, bsl_options=bsl_options)
cross_validate(algo, data, measures=['RMSE','MAE'], cv=5, verbose=False)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the pearson_baseline similarity matrix...
Done computing similarity matrix.


{'test_rmse': array([0.10871431, 0.11922644, 0.12702484, 0.12677312, 0.1010416 ]),
 'test_mae': array([0.07406305, 0.07743698, 0.07805547, 0.07862309, 0.06949558]),
 'fit_time': (0.005767345428466797,
  0.008179903030395508,
  0.007740974426269531,
  0.00646519660949707,
  0.007834196090698242),
 'test_time': (0.007974863052368164,
  0.008294105529785156,
  0.010718345642089844,
  0.007407426834106445,
  0.011068344116210938)}

In [18]:
trainset, testset = train_test_split(data, test_size=0.2)
algo = KNNBaseline(algo = KNNBaseline(sim_options=sim_options, bsl_options=bsl_options))
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.1295


0.1295447753933965

In [19]:
algo.__class__.__name__

'KNNBaseline'

In [20]:
def get_Iu(uid):
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
df_new = pd.DataFrame(predictions, columns=['user_id', 'post_id', 'rui', 'est', 'details'])
df_new['no_item_rated_by_user'] = df_new.user_id.apply(get_Iu)
df_new['no_user_rated_item'] = df_new.post_id.apply(get_Ui)
df_new['error'] = abs(df_new.est - df_new.rui)
df_new.head()

,user_id,post_id,rui,est,details,no_item_rated_by_user,no_user_rated_item,error
0,5d610ae1653a331687083239,5e7bd922cfc8b713f5ac7da9,4.047619,4.012914,"{'actual_k': 0, 'was_impossible': False}",108,1,0.034705
1,5d7c994d5720533e15c3b1e9,5ea5ce9310426255a7aa9b8d,4.523810,4.530255,"{'actual_k': 5, 'was_impossible': False}",71,6,0.006445
2,5ea83e8810426255a7aa9bf2,5e9f40b2a3258347b42f2e45,4.047619,4.213250,"{'actual_k': 0, 'was_impossible': False}",3,2,0.165631
3,5e37b16ceab55d319938a6f8,5e52f2b8d701ab08af792a05,4.047619,4.138721,"{'actual_k': 1, 'was_impossible': False}",20,1,0.091102
4,5e5b4511d701ab08af792b6d,5ea85c1f10426255a7aa9c05,4.285714,4.294059,"{'actual_k': 1, 'was_impossible': False}",4,1,0.008345


In [21]:
best_predictions = df_new.sort_values(by='error')[:5]
worst_predictions = df_new.sort_values(by='error')[-5:]

In [22]:
best_predictions

,user_id,post_id,rui,est,details,no_item_rated_by_user,no_user_rated_item,error
275,5df20f1fee4bb5252b4f5351,5e5196b5d701ab08af7929a8,4.285714,4.285714,"{'actual_k': 1, 'was_impossible': False}",64,1,0.000000
198,5deeef6142a8854bf6eabab9,5e947c4aa3258347b42f283e,4.285714,4.285714,"{'actual_k': 1, 'was_impossible': False}",49,1,0.000000
42,5d60098a653a331687083238,5e7cc074cfc8b713f5ac7db0,4.285714,4.285714,"{'actual_k': 1, 'was_impossible': False}",181,1,0.000000
265,5df20f1fee4bb5252b4f5351,5e52f2b8d701ab08af792a05,4.285714,4.285714,"{'actual_k': 1, 'was_impossible': False}",64,1,0.000000
216,5d7c994d5720533e15c3b1e9,5eb2cbde10426255a7aaa074,4.047619,4.047673,"{'actual_k': 2, 'was_impossible': False}",71,2,0.000054


In [23]:
worst_predictions

,user_id,post_id,rui,est,details,no_item_rated_by_user,no_user_rated_item,error
80,5d60098a653a331687083238,5ddeb6e80eb5e25a8a07f065,4.523810,4.224337,"{'actual_k': 0, 'was_impossible': False}",181,1,0.299473
33,5ed24d5a76027d35905cc7d7,5ed09ad076027d35905cc15f,4.523810,4.207663,{'was_impossible': False},0,8,0.316146
151,5ecb979eeaff6b0c3a58a4f0,5ed175f476027d35905cc562,4.761905,4.409164,{'was_impossible': False},33,0,0.352741
280,5ea890e410426255a7aa9c16,5ea85c1f10426255a7aa9c05,3.571429,4.172950,"{'actual_k': 1, 'was_impossible': False}",4,1,0.601521
276,5ea43f7310426255a7aa9b1d,5e9f3e1aa3258347b42f2e39,3.333333,4.207368,{'was_impossible': False},0,2,0.874035


In [24]:
df_new = pd.merge(df_new,posts,on='post_id')
df_new = df_new[['user_id','post_id','title','category','post_type','rui','est','error']]
df_new

,user_id,post_id,title,category,post_type,rui,est,error
0,5d610ae1653a331687083239,5e7bd922cfc8b713f5ac7da9,What sports will look like in the future,Computer Technology|Robotics|Data Science|Info...,blog,4.047619,4.012914,0.034705
1,5d60098a653a331687083238,5e7bd922cfc8b713f5ac7da9,What sports will look like in the future,Computer Technology|Robotics|Data Science|Info...,blog,4.285714,4.224530,0.061184
2,5d7c994d5720533e15c3b1e9,5ea5ce9310426255a7aa9b8d,Photography,,project,4.523810,4.530255,0.006445
3,5ea83e8810426255a7aa9bf2,5e9f40b2a3258347b42f2e45,Light to fight,Photography,artwork,4.047619,4.213250,0.165631
4,5e37b16ceab55d319938a6f8,5e52f2b8d701ab08af792a05,Network & it's types,Computer Technology|Computer Application|Infor...,blog,4.047619,4.138721,0.091102
...,...,...,...,...,...,...,...,...
285,5df20f1fee4bb5252b4f5351,5dada82f610ba040fbfdf58e,Om Namah Shivaay,Pen and ink,artwork,4.047619,4.322001,0.274382
286,5ecb979eeaff6b0c3a58a4f0,5ed23c8676027d35905cc780,Change.,Calligraphy|Pen and ink,artwork,4.285714,4.409164,0.123450
287,5e1ef04c2a37d20505da2b8b,5eb15a9810426255a7aaa019,MOSAIC ART,Craft|Mixed Media|Conceptual|Mosaic painting|2...,artwork,4.285714,4.218751,0.066963
288,5d7c994d5720533e15c3b1e9,5ea3236810426255a7aa9ac8,Solitude,Photography,artwork,4.047619,4.106518,0.058899


In [25]:
df_new[df_new['user_id']==df_new.user_id.value_counts().index[0]].sort_values(by='error',ascending=True)[:5]

,user_id,post_id,title,category,post_type,rui,est,error
72,5d60098a653a331687083238,5e7cc074cfc8b713f5ac7db0,REAL TALK,Philosophy|Public Philosophy,blog,4.285714,4.285714,0.000000
125,5d60098a653a331687083238,5e9a8437a3258347b42f2c38,Sign of peace,Painting,artwork,4.047619,4.047357,0.000262
210,5d60098a653a331687083238,5ec573a3f2781131cc7e51b8,My First Animated Post.,Art; Science,skill,4.761905,4.759820,0.002085
190,5d60098a653a331687083238,5ecf96e876027d35905cbf46,GAN's Part(2),Computer Technology|Machine Learning,blog,4.285714,4.290266,0.004551
117,5d60098a653a331687083238,5ec57acaf2781131cc7e51d5,Ruins.,Painting,artwork,4.047619,4.054672,0.007053


In [26]:
df_new[df_new['user_id']==df_new.user_id.value_counts().index[-1]].sort_values(by='error',ascending=True)[:5]

,user_id,post_id,title,category,post_type,rui,est,error
193,5ea3f6dc10426255a7aa9aff,5e948db6a3258347b42f28b2,peace,Photography,artwork,4.285714,4.27445,0.011264


In [27]:
test_df = pd.DataFrame(testset,columns = ['user_id','post_id','mixture'])
test_df

,user_id,post_id,mixture
0,5d610ae1653a331687083239,5e7bd922cfc8b713f5ac7da9,4.047619
1,5d7c994d5720533e15c3b1e9,5ea5ce9310426255a7aa9b8d,4.523810
2,5ea83e8810426255a7aa9bf2,5e9f40b2a3258347b42f2e45,4.047619
3,5e37b16ceab55d319938a6f8,5e52f2b8d701ab08af792a05,4.047619
4,5e5b4511d701ab08af792b6d,5ea85c1f10426255a7aa9c05,4.285714
...,...,...,...
285,5d610ae1653a331687083239,5ea8618a10426255a7aa9c0d,4.047619
286,5e1ef04c2a37d20505da2b8b,5eb15a9810426255a7aaa019,4.285714
287,5d610ae1653a331687083239,5e8c2d01a3258347b42f2627,4.285714
288,5d7c994d5720533e15c3b1e9,5ea3236810426255a7aa9ac8,4.047619


In [28]:
def get_n_recommend(user_id,n=5):
    test_results = pd.DataFrame(columns=['user_id','post_id','estimate'])
    for i in test_df.post_id.unique():
        tmp = pd.DataFrame([[user_id,i,algo.predict(user_id,i)[3]]] , columns=['user_id','post_id','estimate'])
        test_results = test_results.append(tmp,ignore_index=True)
    test_results = pd.merge(test_results,posts,on = 'post_id')
    return test_results.sort_values(by='estimate',ascending=False).reset_index(drop=True)[:n]

In [29]:
get_n_recommend(test_df.loc[0,'user_id'],10)

,user_id,post_id,estimate,title,category,post_type
0,5d610ae1653a331687083239,5e95ed0aa3258347b42f2a0e,4.619318,Lovely explosion box!,Craft work,skill
1,5d610ae1653a331687083239,5e9db770a3258347b42f2dcf,4.605742,E s c a p e,Photography,skill
2,5d610ae1653a331687083239,5ec573a3f2781131cc7e51b8,4.546860,My First Animated Post.,Art; Science,skill
3,5d610ae1653a331687083239,5ec2f7c074f7660d73aa116e,4.534611,Making of Daaku Art,Art,skill
4,5d610ae1653a331687083239,5ec54fc9f2781131cc7e50f5,4.531119,Future Communication Predictions,Science;Technology,skill
5,5d610ae1653a331687083239,5ec57914f2781131cc7e51c8,4.505427,The Power of Indian Audience.,Art,skill
6,5d610ae1653a331687083239,5ebd5b46514aab59896bcd5a,4.493530,Quick Sketch of Gangster Skull.,Sketch Video,skill
7,5d610ae1653a331687083239,5ec3bc1a74f7660d73aa1207,4.452661,The Making of Daaku,Art,skill
8,5d610ae1653a331687083239,5ed13d2876027d35905cc4c2,4.440505,Launching an EC2 instance.,Technology,skill
9,5d610ae1653a331687083239,5e7de250a3258347b42f210a,4.409700,Top 10 big benefits of having a digital market...,Digital Marketing,skill
